In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
from random import shuffle
import string

Dans l'ordre :

- Importation de la bibliothèque NumPy, utilisée pour la manipulation de tableaux et matrices.
- Importation de Keras, une interface de haut niveau pour les réseaux de neurones, ici utilisée avec TensorFlow comme backend.
- Importation des couches (layers) de Keras, utilisées pour construire des réseaux de neurones.
- Importation de la classe Path de la bibliothèque pathlib, pour une manipulation aisée des chemins de fichiers.
- Importation de la fonction shuffle du module random, pour mélanger des données ou listes.
- Importation du module string, souvent utilisé pour des opérations sur les chaînes de caractères.

In [ ]:
NUM_CLASSES = 10  
DATA_PATH = "notMNIST_small/"  
INPUT_SHAPE = (28, 28, 1)  
BATCH_SIZE = 128  
EPOCHS = 25  
TRAINING_SPLIT = 0.8

NUM_CLASSES détermine le nombre de catégories différentes à classer, comme les lettres de A à J. DATA_PATH spécifie l'emplacement des données d'entraînement, dans ce cas, un dossier nommé "notMNIST_small/". INPUT_SHAPE définit la taille et la forme des images d'entrée que le modèle va traiter, ici des images de 28x28 pixels avec un seul canal de couleur (niveaux de gris). BATCH_SIZE et EPOCHS sont des paramètres d'entraînement : le premier indique combien d'exemples sont traités ensemble lors de l'apprentissage, et le second combien de fois l'ensemble du dataset est parcouru pendant l'entraînement. Enfin, TRAINING_SPLIT détermine la proportion du dataset à utiliser pour l'entraînement, le reste pouvant être utilisé pour la validation.

In [ ]:

training_data = []
letters = string.ascii_uppercase[:NUM_CLASSES]

for letter in letters:
    print(f"Loading letter {letter}... ", end="")
    for file in Path(DATA_PATH + letter).glob("*.png"):
        img = keras.preprocessing.image.load_img(file, color_mode="grayscale", target_size=INPUT_SHAPE)
        img_array = keras.preprocessing.image.img_to_array(img)
        training_data.append((letter, img_array))
print(f"Total images loaded: {len(training_data)}")


On commence par créer une liste vide training_data pour stocker les données. Ensuite, on génère une liste de lettres à charger (letters), limitée par le nombre de classes spécifié dans NUM_CLASSES.Pour chaque lettre dans cette liste, le code parcourt tous les fichiers .png le dossier. Pour chaque fichier image trouvé, on charge l'image en niveaux de gris (color_mode="grayscale") et on la redimensionne selon les dimensions spécifiées dans INPUT_SHAPE. L'image est ensuite convertie en un tableau numpy.Après la conversion, chaque image et son étiquette de lettre correspondante sont ajoutées à la liste training_data. Finalement, après avoir chargé toutes les images, on affiche le nombre total d'images chargées. 

In [ ]:
shuffle(training_data)

training_set, evaluation_set = train_test_split(training_data, test_size=1 - TRAINING_SPLIT)

training_labels = np.array([ord(item[0]) - ord('A') for item in training_set])
training_images = np.array([item[1] for item in training_set])
evaluation_labels = np.array([ord(item[0]) - ord('A') for item in evaluation_set])
evaluation_images = np.array([item[1] for item in evaluation_set])

training_images = training_images.astype("float32") / 255
evaluation_images = evaluation_images.astype("float32") / 255

training_images = np.expand_dims(training_images, -1)
evaluation_images = np.expand_dims(evaluation_images, -1)

training_labels = to_categorical(training_labels, NUM_CLASSES)
evaluation_labels = to_categorical(evaluation_labels, NUM_CLASSES)


On commence par mélanger les données d'entraînement pour assurer une distribution aléatoire. Ensuite, on utilise la fonction train_test_split pour diviser le jeu de données en deux parties : un ensemble d'entraînement et un ensemble d'évaluation, avec la proportion déterminée par la variable TRAINING_SPLIT. Pour chaque ensemble, on sépare les étiquettes (représentant les lettres) des images. Les étiquettes sont converties de caractères en valeurs numériques correspondant à leur position dans l'alphabet. Les images sont ensuite normalisées pour que leurs valeurs de pixels soient comprises entre 0 et 1. Après cela, les images sont redimensionnées pour s'assurer qu'elles ont la forme attendue par le modèle, et enfin, les étiquettes sont converties en matrices de classe binaire, format requis pour la classification multiclasse dans Keras.

In [ ]:
cnn_model = keras.Sequential([
    keras.Input(shape=INPUT_SHAPE),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

On commence par spécifier la forme d'entrée des données (INPUT_SHAPE), qui correspond aux dimensions des images à traiter. Le modèle est construit en empilant plusieurs couches : des couches de convolution (Conv2D) avec des activations ReLU (une fonction d'activation non linéaire couramment utilisée) et un padding 'same' pour conserver les dimensions spatiales après la convolution. Chaque groupe de couches de convolution est suivi d'une couche de max pooling (MaxPooling2D), qui réduit la dimensionnalité spatiale et aide à extraire les caractéristiques les plus saillantes. Après plusieurs de ces groupes de couches de convolution et de pooling, la couche Flatten transforme les données en un vecteur unidimensionnel pour les couches denses suivantes. La couche Dense de 256 neurones sert de couche de classification entièrement connectée, suivie d'une couche Dropout pour la régularisation (réduire le surajustement). Enfin, la dernière couche Dense avec l'activation softmax produit une distribution de probabilité sur les classes possibles (définies par NUM_CLASSES), permettant au modèle de prédire la classe de chaque image d'entrée.

In [ ]:
cnn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

cnn_model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1)

score = model.evaluate(evaluate_images, evaluate_labels, verbose=0)

print("Loss score :", score[0])
print("Accuracy score :", score[1])

Tout d'abord, model.compile prépare le modèle pour l'entraînement en définissant la fonction de perte (loss) comme "categorical_crossentropy", l'optimiseur comme "adam" et la métrique d'évaluation comme "accuracy" (pour mesurer la précision de la classification). Ensuite, model.fit entraîne le modèle sur les images d'entraînement (train_images) avec leurs étiquettes correspondantes (train_labels), en utilisant la taille de lot spécifiée (BATCH_SIZE) et le nombre d'époques (EPOCHS). Un pourcentage des données d'entraînement (10% ici, spécifié par validation_split=0.1) est utilisé comme ensemble de validation pour surveiller la performance du modèle sur des données non vues pendant l'entraînement. Après l'entraînement, model.evaluate est utilisé pour tester la performance du modèle sur un ensemble de données d'évaluation distinct, donnant le score de perte et la précision, qui sont ensuite imprimés. Ce processus est essentiel pour comprendre comment le modèle performe et où il peut être amélioré.
